In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

#### Sequential API

In [4]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [5]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [6]:
model = keras.Sequential(
    [
        layers.Input(shape=(32,32,3)),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(pool_size = (2,2)),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10)
    ]
)

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 8, 8, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 8192)              0

In [8]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

r = model.fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
1563/1563 [==============================] - 15s 7ms/step - loss: 1.3949 - accuracy: 0.4977 - val_loss: 1.1091 - val_accuracy: 0.6055
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.9753 - accuracy: 0.6588 - val_loss: 0.9087 - val_accuracy: 0.6835
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 0.7986 - accuracy: 0.7211 - val_loss: 0.8351 - val_accuracy: 0.7085
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 0.6900 - accuracy: 0.7606 - val_loss: 0.7968 - val_accuracy: 0.7269
Epoch 5/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.5904 - accuracy: 0.7932 - val_loss: 0.8098 - val_accuracy: 0.7338
Epoch 6/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.5117 - accuracy: 0.8216 - val_loss: 0.7901 - val_accuracy: 0.7377
Epoch 7/10
1563/1563 [==============================] - 12s 8ms/step - loss: 0.4322 - accuracy: 0.8488 - val_loss: 0.8942 - val_ac

#### Functional API

In [9]:
def my_model():
    inputs = layers.Input((32,32,3))
    x = layers.Conv2D(32, 3, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Conv2D(128, 5, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(64, use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    outputs = layers.Dense(10)(x)
    model = keras.Model(inputs, outputs)
    return model

In [10]:
model = my_model()

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 16, 16, 32)        0     

In [12]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    metrics = ['accuracy']
)

r = model.fit(
    X_train, y_train,
    batch_size = 32,
    epochs = 10,
    validation_data=(X_test, y_test)
)

Epoch 1/10
1563/1563 [==============================] - 24s 15ms/step - loss: 1.2806 - accuracy: 0.5441 - val_loss: 1.1475 - val_accuracy: 0.5930
Epoch 2/10
1563/1563 [==============================] - 25s 16ms/step - loss: 0.9482 - accuracy: 0.6669 - val_loss: 1.3588 - val_accuracy: 0.5496
Epoch 3/10
1563/1563 [==============================] - 27s 18ms/step - loss: 0.8127 - accuracy: 0.7153 - val_loss: 1.4177 - val_accuracy: 0.5516
Epoch 4/10
1563/1563 [==============================] - 38s 25ms/step - loss: 0.7269 - accuracy: 0.7462 - val_loss: 0.8599 - val_accuracy: 0.7032
Epoch 5/10
1563/1563 [==============================] - 35s 22ms/step - loss: 0.6541 - accuracy: 0.7697 - val_loss: 1.1602 - val_accuracy: 0.6405
Epoch 6/10
1563/1563 [==============================] - 34s 22ms/step - loss: 0.5971 - accuracy: 0.7895 - val_loss: 0.7762 - val_accuracy: 0.7380
Epoch 7/10
1563/1563 [==============================] - 48s 31ms/step - loss: 0.5359 - accuracy: 0.8133 - val_loss: 0.9430 -